<a href="https://colab.research.google.com/github/winneking/hello-world/blob/master/augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import cv2
import os
import math
import h5py as h5py


In [0]:
!apt-get -qq install -y graphviz && pip install -q pydot
import pydot
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model
from IPython.display import Image

from keras.utils.data_utils import get_file
from keras.models import Sequential, Model
from keras.layers import Input,Dense,Dropout,Flatten
from keras.layers.convolutional import Conv2D,MaxPooling2D
from kera.applications.vgg16 import VGG16
from keras.optimizers import SGD

from keras.preprocessing.image import ImageDataGernerator
from keras.preprocessing import image

RATIO = 0.2

NUM_DENSE = 5

epochs = 1

ishape=48

def tran_y(y):
    y_ohe = np.zeros(NUM_DENSE)
    y_ohe[y] = 1
    return y_ohe
  
print('download vgg Model')

model_vgg = VGG16(include_top = False, weights = 'imagenet',input_shape = (ishape,ishape, 3))

for layer in model_vgg.layers:
          layer.trainable = False
model = Flatten()(model_vgg.output)
model = Dense(4096, activation='relu',name='fc1')(model)
model = Dense(4096, activation='relu',name='fc2')(model)
model = Dropout(0.5)(model)
model = Dense(NUM_DENSE,activation = 'softmax', name= 'prediction')(model)
model_vgg_pretrain = Model(model_vgg.input, model, name = 'vgg16_pretrain')

print('Model compile')

sgd = SGD(lr = 0.05, decay = 1e-5)
model_vgg_pretrain.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])
